In [ ]:
# Bajar datos

# Varios trimestres
# Existen version en .csv, .dbf, .dta. .sav

#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2025_trim2_dta.zip # 2025_2
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2023_trim4_dbf.zip
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim4_dbf.zip
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim1_dta.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2021_trim4_dta.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2021_trim3_dta.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2020trim1_dbf.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2019trim4_dbf.zip
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2015trim1_dbf.zip
#https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2010trim1_dbf.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2005trim2_dbf.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2005trim1_dbf.zip

########
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2025_trim2_dta.zip
# https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2022_trim4_dta.zip




In [8]:
import pandas as pd
import requests 
from tqdm import tqdm # Importamos tqdm



In [9]:
url = 'https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2025_trim2_dta.zip'
savename = "Data/enoe_2025_trim2_dta.zip"

In [10]:

print(f"Descargando el archivo: {savename}")

try:
    # Hacemos la petición GET a la URL, con stream=True para descargar por partes
    with requests.get(url, stream=True) as r:
        # Verificamos que la petición fue exitosa (código 200)
        r.raise_for_status()
        
        # Abrimos un archivo local en modo de escritura binaria ('wb')
        with open(savename, 'wb') as f:
            # Iteramos sobre el contenido de la descarga en trozos de 8KB
            for chunk in r.iter_content(chunk_size=8192): 
                # Escribimos cada trozo en el archivo local
                f.write(chunk)
                
    print("¡Descarga completada con éxito!")

except requests.exceptions.HTTPError as errh:
    print(f"Error HTTP: {errh}")
except requests.exceptions.ConnectionError as errc:
    print(f"Error de Conexión: {errc}")
except requests.exceptions.Timeout as errt:
    print(f"Error de Timeout: {errt}")
except requests.exceptions.RequestException as err:
    print(f"¡Oops! Algo salió mal: {err}")

Descargando el archivo: Data/enoe_2025_trim2_dta.zip
¡Descarga completada con éxito!


In [ ]:
import os
import requests
import zipfile

def get_url(year, quarter):
    """
    Construye la URL de descarga para un año y trimestre dados.
    Maneja los diferentes formatos de URL del INEGI.
    """
    base_url = "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/"

    # 2005T1 to 2021T1
    if (2005 <= year <= 2021 and quarter >= 1 and (year, quarter) != (2020, 1) and (year, quarter) != (2020, 2)):
        if year == 2021 and quarter > 1:
            pass  # Pasa a la siguiente condición para 2021 T2 y en adelante
        else:
            filename = f"{year}trim{quarter}_dbf.zip"
            return base_url + filename

    # 2020T3 to 2022T4
    if (year == 2020 and quarter >= 3) or (2021 <= year <= 2022):
        if (year == 2021 and quarter == 1):
             filename = f"{year}trim{quarter}_dbf.zip"
             return base_url + filename

        filename = f"enoe_n_{year}_trim{quarter}_dta.zip"
        return base_url + filename

    # 2023T1 and after
    if year >= 2023:
        # INEGI usa .dbf o .dta, se prueban ambos.
        filename_dta = f"enoe_n_{year}_trim{quarter}_dta.zip"
        filename_dbf = f"enoe_n_{year}_trim{quarter}_dbf.zip"
        
        # Se prueba el formato dta primero
        url_dta = base_url + filename_dta
        response = requests.head(url_dta)
        if response.status_code == 200:
            return url_dta

        # Si no existe, se prueba el formato dbf
        url_dbf = base_url + filename_dbf
        response = requests.head(url_dbf)
        if response.status_code == 200:
            return url_dbf

    return None

def download_and_extract_data(start_year=2005, start_quarter=1):
    """
    Descarga y organiza las bases de datos de la ENOE.
    
    Args:
        start_year (int): El año de inicio.
        start_quarter (int): El trimestre de inicio.
    """
    
    # Directorios base
    temp_dir = "Data/temporal"
    main_data_dir = "Data/ENOE"
    
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(main_data_dir, exist_ok=True)
    
    current_year = 2005
    current_quarter = 1

    # Bucle para iterar a través de los años y trimestres
    while current_year <= 2023: # Puedes ajustar el año final
        if current_year < start_year or (current_year == start_year and current_quarter < start_quarter):
            current_quarter += 1
            if current_quarter > 4:
                current_quarter = 1
                current_year += 1
            continue

        # Casos especiales sin datos
        if (current_year == 2020 and current_quarter in [1, 2]):
            print(f"Saltando {current_year} T{current_quarter}: No hay datos.")
            current_quarter += 1
            if current_quarter > 4:
                current_quarter = 1
                current_year += 1
            continue

        url = get_url(current_year, current_quarter)
        if url:
            file_name = url.split('/')[-1]
            temp_file_path = os.path.join(temp_dir, file_name)
            
            # Nombre de la carpeta de destino
            dest_dir = os.path.join(main_data_dir, f"ENOE{current_year}{current_quarter}")
            os.makedirs(dest_dir, exist_ok=True)
            
            print(f"Descargando {file_name} de {url}")
            
            try:
                with requests.get(url, stream=True) as r:
                    r.raise_for_status()
                    with open(temp_file_path, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                
                # Descomprimir el archivo
                with zipfile.ZipFile(temp_file_path, 'r') as zip_ref:
                    zip_ref.extractall(dest_dir)
                    print(f"  -> Descomprimido en {dest_dir}")
                
            except requests.exceptions.RequestException as e:
                print(f"  -> Error al descargar {file_name}: {e}")
            except zipfile.BadZipFile:
                print(f"  -> Error: El archivo {file_name} está corrupto o no es un archivo zip válido.")
            
        current_quarter += 1
        if current_quarter > 4:
            current_quarter = 1
            current_year += 1
            
if __name__ == "__main__":
    # Puedes cambiar los valores para iniciar desde un año y trimestre específicos
    start_year = 2005
    start_quarter = 1
    download_and_extract_data(start_year, start_quarter)

Descargando 2005trim1_dbf.zip de https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2005trim1_dbf.zip
  -> Descomprimido en Data/ENOE\ENOE20051
Descargando 2005trim2_dbf.zip de https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2005trim2_dbf.zip
  -> Descomprimido en Data/ENOE\ENOE20052
Descargando 2005trim3_dbf.zip de https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2005trim3_dbf.zip
  -> Descomprimido en Data/ENOE\ENOE20053
Descargando 2005trim4_dbf.zip de https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2005trim4_dbf.zip
  -> Descomprimido en Data/ENOE\ENOE20054
Descargando 2006trim1_dbf.zip de https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2006trim1_dbf.zip
  -> Error al descargar 2006trim1_dbf.zip: ('Connection broken: IncompleteRead(2466927 bytes read, 28464759 more expected)', IncompleteRead(2466927 bytes read, 28464759 more expected))
Descargando 2006trim2_dbf.zip de https://www.in

In [ ]:
# Homologar     

In [ ]:
# Estadística Exploratoria
